In [23]:
from csv import DictReader
import osmnx as ox
import networkx as nx
import numpy as np
from datetime import datetime

In [24]:
file = open("punkty5.txt", 'r')
reader = DictReader(file)
points = list()

for dictionary in reader:
    points.append(dictionary)
print(points)

[{'adres': 'adres1', 'y': '23.141401', 'x': '53.142928'}, {'adres': 'adres2', 'y': '23.150168', 'x': '53.125021'}, {'adres': 'adres3', 'y': '23.094750', 'x': '53.157217'}, {'adres': 'adres4', 'y': '23.095895', 'x': '53.149964'}, {'adres': 'adres5', 'y': '23.198124', 'x': '53.106625'}]


In [25]:
ox.settings.log_console = True
ox.settings.use_cache = True

In [26]:
miejsce = "Białystok, Polska"
tryb = "drive"
kryterium = "length"

In [27]:
graph = ox.graph_from_place(miejsce, network_type=tryb )

In [28]:
distance_matrix = np.empty((len(points),len(points)))

In [29]:
for i in range(len(points)):
    origin_node = ox.nearest_nodes(graph, float(points[i]['y']), float(points[i]['x']))
    for j in range(len(points)):
        destination_node = ox.nearest_nodes(graph, float(points[j]['y']), float(points[j]['x']))
        distance_matrix[i][j] = nx.shortest_path_length(graph, origin_node, destination_node, weight = kryterium)
distance_matrix

array([[    0.   ,  3378.35 ,  4226.012,  4238.483,  7355.548],
       [ 2521.995,     0.   ,  6108.307,  6094.567,  4748.549],
       [ 4324.323,  6606.536,     0.   ,  1129.315, 10583.734],
       [ 4256.3  ,  6444.962,  1129.315,     0.   , 10422.16 ],
       [ 6728.663,  4356.176, 10314.975, 10301.235,     0.   ]])

In [30]:
import gen_algorithms as ga

In [31]:
#Hyperparameters:
number_of_generations = 5
population_size = 10
number_of_parents = 6
length = len(points)
beginning = 1
number_of_offspring = population_size - number_of_parents
mutation_rate = 10 #%

In [32]:
#Populacja początkowa:
start_time = datetime.now()
population = []
i=0
while i < population_size:
    ind = ga.create_individual(beginning, length)
    if ind not in population:
        population.append(ind)
        i+=1
max_individuals=[]
print(f'Population: {population}')

for generation in range(number_of_generations):

    fitness_list=[]
    for indiv in population:
        fitness_list.append(ga.calculate_fitness(indiv, distance_matrix, length))
    max_individual = max(fitness_list)
    max_individuals.append(max_individual)
    print(f'fitness: {fitness_list}')
    #print(f'Generation: {generation}, max fitness: {max_individual}')

    parents = ga.select_parents(population, fitness_list, number_of_parents)
    print(f'Parents: {parents}')

    children = ga.crossover(parents, number_of_offspring, length)
    print(f'Children: {children}')

    for child in children:
        ga.mutation(child, mutation_rate)
    
    population = parents + children
end_time = datetime.now()

Population: [[1, 2, 3, 0, 4], [1, 4, 3, 2, 0], [1, 0, 4, 3, 2], [1, 2, 0, 4, 3], [1, 4, 2, 0, 3], [1, 4, 3, 0, 2], [1, 3, 4, 2, 0], [1, 0, 2, 4, 3], [1, 3, 2, 4, 0], [1, 3, 2, 0, 4]]
Parents: [[1, 2, 3, 0, 4], [1, 3, 2, 0, 4], [1, 4, 3, 2, 0], [1, 0, 4, 3, 2], [1, 3, 2, 4, 0], [1, 4, 2, 0, 3]]
Children: [[1, 0, 4, 3, 2], [1, 2, 3, 0, 4], [1, 3, 2, 4, 0], [1, 3, 2, 0, 4]]
Parents: [[1, 2, 3, 0, 4], [1, 2, 3, 0, 4], [1, 3, 2, 0, 4], [1, 3, 2, 0, 4], [1, 4, 3, 2, 0], [1, 0, 4, 3, 2]]
Children: [[1, 4, 3, 2, 0], [1, 2, 3, 4, 0], [1, 3, 2, 0, 4], [1, 0, 4, 3, 2]]
Parents: [[1, 2, 3, 0, 4], [1, 2, 3, 0, 4], [1, 3, 2, 0, 4], [1, 3, 2, 0, 4], [1, 3, 2, 0, 4], [1, 4, 3, 2, 0]]
Children: [[1, 3, 2, 0, 4], [1, 2, 3, 0, 4], [1, 3, 2, 0, 4], [1, 2, 3, 0, 4]]
Parents: [[1, 2, 3, 0, 4], [1, 2, 3, 0, 4], [1, 2, 3, 0, 4], [1, 2, 3, 0, 4], [1, 3, 2, 0, 4], [1, 3, 2, 0, 4]]
Children: [[1, 3, 2, 0, 4], [1, 2, 3, 0, 4], [1, 2, 3, 0, 4], [1, 3, 2, 0, 4]]
Parents: [[1, 2, 3, 0, 4], [1, 2, 3, 0, 4], [1, 2, 3,

In [33]:
duration = end_time - start_time
print(f'Duration: {duration} \n\n')
difference = max_individuals[0] - max_individuals[-1]
improvement = (difference/max_individuals[0])*100
print(f'first generation: {max_individuals[0]}, last_generation: {max_individuals[-1]}')
print(f'difference: {difference}, improvement: {improvement} %')

best = max(fitness_list)
best_route_index = fitness_list.index(best)
best_route = population[best_route_index]
f=open("Solution.txt", 'w')
f.write(f'Duration: {duration}\n')
f.write(f'first generation: {-max_individuals[0]}, \nlast_generation: {-max_individuals[-1]}\n')
f.write(f'difference: {-difference}, \nimprovement: {improvement} %\n')
f.write(f'best route: {best_route}\n\n')
f.write(f'{duration}')
f.write('\n')
f.write(f'{-max_individuals[0]}')
f.write('\n')
f.write(f'{-max_individuals[-1]}')
f.write('\n')
f.write(f'{improvement}')
f.close()

Duration: 0:00:00.001005 


first generation: -23205.645999999997, last_generation: -23205.645999999997
difference: 0.0, improvement: -0.0 %
